EIT on pancake

In [125]:
from qm.qua import *
from qm import QuantumMachinesManager, SimulationConfig
from quam.components import BasicQuAM, SingleChannel
from quam.components.pulses import SquarePulse
from qualang_tools.units import unit
from QuAM_utilities import *

import matplotlib.pyplot as plt
import time

u = unit()

In [126]:
machine = BasicQuAM()

# AOM RFs
cool_double_rf = 200*u.MHz
repump_double_rf = 200*u.MHz

In [127]:
machine.channels['AOM_single_repumper'] = AOM_single_repumper = SingleChannel(opx_output=('con1', 1), intermediate_frequency=200e6)
machine.channels['AOM_double_repumper'] = AOM_double_repumper = SingleChannel(opx_output=('con1', 2), intermediate_frequency=repump_double_rf)
machine.channels['AOM_single_cooler'] = AOM_single_cooler = SingleChannel(opx_output=('con1', 3), intermediate_frequency=200e6)
machine.channels['AOM_double_cooler'] = AOM_double_cooler = SingleChannel(opx_output=('con1', 4), intermediate_frequency=cool_double_rf)

In [128]:
repump_square_amplitude_EIT_single = 0.03 # 0.03
cool_square_amplitude_EIT_single = 0.1 # 0.1

repump_square_amplitude_heating_single = 0.38
cool_square_amplitude_heating_single = 0.38

cool_square_amplitude_double = 0.38
repump_square_amplitude_double = 0.38


heating_time = 200*u.us
EIT_pulse_time = 1*u.ms


## Heating pulses
AOM_single_cooler.operations['cooler_high_pulse_heating_single'] = SquarePulse(length=round(heating_time/4)*4, amplitude=cool_square_amplitude_heating_single)
AOM_double_cooler.operations['cooler_high_pulse_heating_double'] = SquarePulse(length=round(heating_time/4)*4, amplitude=cool_square_amplitude_double)

AOM_single_repumper.operations['repump_high_pulse_heating_single'] = SquarePulse(length=round(heating_time/4)*4, amplitude=repump_square_amplitude_heating_single)
AOM_double_repumper.operations['repump_high_pulse_heating_double'] = SquarePulse(length=round(heating_time/4)*4, amplitude=repump_square_amplitude_double)

## EIT pulses
AOM_single_cooler.operations['cooler_high_pulse_EIT_single'] = SquarePulse(length=round(EIT_pulse_time/4)*4, amplitude=cool_square_amplitude_EIT_single)
AOM_double_cooler.operations['cooler_high_pulse_EIT_double'] = SquarePulse(length=round(EIT_pulse_time/4)*4, amplitude=cool_square_amplitude_double)

AOM_single_repumper.operations['repump_high_pulse_EIT_single'] = SquarePulse(length=round(EIT_pulse_time/4)*4, amplitude=repump_square_amplitude_EIT_single)
AOM_double_repumper.operations['repump_high_pulse_EIT_double'] = SquarePulse(length=round(EIT_pulse_time/4)*4, amplitude=repump_square_amplitude_double)

In [129]:
qua_config = machine.generate_config()

Write the QUA program

Modify the utilities script in order to handle the time parameter?

In [130]:
with program() as EIT:
    
    with infinite_loop_():
        
        wait_for_trigger('AOM_double_repumper')
        wait_for_trigger('AOM_double_cooler')
        wait_for_trigger('AOM_single_repumper')
        wait_for_trigger('AOM_single_cooler')

        # # Heating pulses
        # play('cooler_high_pulse_heating_single', 'AOM_single_cooler')
        # play('cooler_high_pulse_heating_double', 'AOM_double_cooler')
        # play('repump_high_pulse_heating_single', 'AOM_single_repumper')
        # play('repump_high_pulse_heating_double', 'AOM_double_repumper')
        
        # EIT pulses
        play('cooler_high_pulse_EIT_single', 'AOM_single_cooler')
        play('cooler_high_pulse_EIT_double', 'AOM_double_cooler')
        play('repump_high_pulse_EIT_single', 'AOM_single_repumper')
        play('repump_high_pulse_EIT_double', 'AOM_double_repumper')

Open the machine, load the state and run the program

In [131]:
ip_address = '130.79.148.167'
qmm = QuantumMachinesManager(host=ip_address)

2024-12-19 10:44:24,868 - qm - INFO     - Performing health check
2024-12-19 10:44:24,879 - qm - INFO     - Health check passed


In [132]:
%matplotlib qt

Simulation = False

if Simulation == True:

    start_time = time.time()
    simulated_job = qmm.simulate(qua_config, EIT, SimulationConfig(duration=int(400*1e3/4)))
    end_time = time.time()
    print(f"Simulation time: {end_time - start_time} s")

    # get DAC and digital samples
    samples = simulated_job.get_simulated_samples()
    plt.figure()
    plt.plot(samples.con1.analog["1"], "--", label="sg_repumper")
    plt.plot(samples.con1.analog["2"], "-", label="db_repumper")
    #plt.plot(samples.con1.analog["3"], "--", label="sg_cooler")
    #plt.plot(samples.con1.analog["4"], "-", label="db_cooler")
    plt.plot(samples.con1.analog["5"], "-", label="tweezer_mod")
    #plt.plot(samples.con1.analog["6"], "-", label="tweezer_beat")
    plt.legend()
    plt.xlabel("Time [ns]")
    plt.ylabel("Signal [V]")        
    plt.show()
else:
    qm = qmm.open_qm(qua_config)
    job = qm.execute(EIT)

2024-12-19 10:44:25,002 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing Quantum Machines: 
2024-12-19 10:44:25,003 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1734601360399 is canceling running job 1734512114420.
2024-12-19 10:44:25,036 - qm - INFO     - Sending program to QOP for compilation
2024-12-19 10:44:25,145 - qm - INFO     - Executing program


In [133]:
job.halt()

True